In [ ]:
import pandas as pd
import numpy as np
import random


In [ ]:
file_path = 'ml-1m/ml-1m/movies.dat' 

try:
    df_movies = pd.read_csv(file_path, sep='::',
                     encoding='latin-1', names=['MovieID', 'Title', 'Genres'], engine='python')
    print("Fichier lu avec succès!")

except Exception as e:
    print(f"Erreur lors de la lecture du fichier : {e}")

def clean_movie_data(df_input):
    df_input.columns = ["MovieID", "Title", "Genres"]
    df_input['Year'] = df_input['Title'].str.extract(r'\((\d{4})\)', expand=False)
    df_input['Title'] = df_input['Title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)
    df_input['Genres'] = df_input['Genres'].apply(lambda x: x.split('|'))
    df_input.set_index("MovieID", inplace=True)

    return df_input

df_final = clean_movie_data(df_movies)

df_final.to_csv("movies_full_cleaned.csv")
print(df_final.head())

Fichier lu avec succès!
                               Title                            Genres  Year
MovieID                                                                     
1                          Toy Story   [Animation, Children's, Comedy]  1995
2                            Jumanji  [Adventure, Children's, Fantasy]  1995
3                   Grumpier Old Men                 [Comedy, Romance]  1995
4                  Waiting to Exhale                   [Comedy, Drama]  1995
5        Father of the Bride Part II                          [Comedy]  1995


In [ ]:


df_ratings = pd.read_csv(
    'ml-1m/ml-1m/ratings.dat',
    sep='::',
    engine='python',      
    names=["UserID", "MovieID", "Rating", "Timestamp"],
    encoding='latin-1'
)

df_ratings.set_index("MovieID", inplace=True)

print(df_ratings.head())

         UserID  Rating  Timestamp
MovieID                           
1193          1       5  978300760
661           1       3  978302109
914           1       3  978301968
3408          1       4  978300275
2355          1       5  978824291


In [ ]:
df_merged = pd.merge(df_movies, df_ratings, on='MovieID').reset_index()
print(df_merged.head())

   MovieID      Title                           Genres  Year  UserID  Rating  \
0        1  Toy Story  [Animation, Children's, Comedy]  1995       1       5   
1        1  Toy Story  [Animation, Children's, Comedy]  1995       6       4   
2        1  Toy Story  [Animation, Children's, Comedy]  1995       8       4   
3        1  Toy Story  [Animation, Children's, Comedy]  1995       9       5   
4        1  Toy Story  [Animation, Children's, Comedy]  1995      10       5   

   Timestamp  
0  978824268  
1  978237008  
2  978233496  
3  978225952  
4  978226474  


In [ ]:

class MovieRecommender:
    def __init__(self, df, epsilon=0.1, decay=True):
        """
        df : DataFrame contenant au moins ['MovieID', 'Rating']
        epsilon : valeur initiale de epsilon
        decay : si True, epsilon_t = epsilon / t
        """
        self.df = df
        self.movie_ids = df['MovieID'].unique().tolist()
        self.K = len(self.movie_ids)

        self.epsilon_0 = epsilon
        self.epsilon = epsilon
        self.decay = decay
        self.t = 0

        # Q-values 
        self.q_values = {m_id: 0.0 for m_id in self.movie_ids}

        self.attempts = {m_id: 0 for m_id in self.movie_ids}

    def select_movie(self):
        """Choix du film selon ε-greedy"""
        if random.random() < self.epsilon:
            # Exploration
            return random.choice(self.movie_ids)
        else:
            # Exploitation
            return max(self.q_values, key=self.q_values.get)

    def get_reward(self, movie_id):
        """
        Simule une récompense : tirage aléatoire
        d'une note existante pour ce film
        """
        return self.df[self.df['MovieID'] == movie_id]['Rating'].sample(1).values[0]

    def update_scores(self, movie_id, reward):
        """Mise à jour incrémentale de la moyenne"""
        self.attempts[movie_id] += 1
        n = self.attempts[movie_id]
        old_q = self.q_values[movie_id]
        self.q_values[movie_id] = old_q + (reward - old_q) / n

    def step(self):
        """Une itération complète du bandit"""
        self.t += 1

        # Mise à jour de epsilon
        if self.decay:
            self.epsilon = self.epsilon_0 / self.t

        movie = self.select_movie()
        reward = self.get_reward(movie)
        self.update_scores(movie, reward)

        return movie, reward

    def best_movie(self):
        """Film avec la meilleure récompense estimée"""
        return max(self.q_values, key=self.q_values.get)


In [ ]:
recommender = MovieRecommender(df_merged, epsilon=1.0, decay=True)

T = 5000
for _ in range(T):
    recommender.step()

best_movie = recommender.best_movie()
best_movie, recommender.q_values[best_movie]


(3328, np.float64(3.8628514056224863))